In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

shift = np.array([-5.27, 3.21])

tilt = np.array([[-0.8563,  0.5165],
                 [0.5165,  0.8563]])

In [ ]:
# READ THE STATISTICAL ERROR ESTIMATES ----------------

folder_name = ""

loc_mod = np.loadtxt(folder_name + "central_values_bins.dat")
cov_mod = np.loadtxt(folder_name + "cov_mat_bins_with_stretch.dat")
cov_mod = cov_mod.reshape(nbins, 2, 2)


In [ ]:
# COMPUTATION OF p-values  FOR STATISTICAL ERROR ESTIMATES ----------------
# 1. statistical_central value calculation
coadd_cov_mod = np.linalg.inv(np.sum(np.linalg.inv(cov_mod), 0))

temp = np.zeros(2)
for i in range(7):
    temp += np.linalg.inv(cov_mod[i]).dot(loc_mod[i])
coadd_loc_mod = coadd_cov_mod.dot(temp)

# p-value estimation
print("p-values (statistical):")
for i in range(7):
    Z = np.linalg.cholesky(cov_mod[i])
    Xprime = np.linalg.inv(Z).dot(coadd_loc_mod - loc_mod[i])
    print("bin {} = {}".format((i+1), np.exp(-0.5 *
                               np.linalg.norm(Xprime) ** 2)))
# -------------------------------------------------------------------------

In [ ]:
# STATISTICAL ESTIMATES IN T0-GAMMA SPACE ----------------

# Convert to ln tau0 - gamma space
loc_orig = xfm(loc_mod, shift, tilt)

cov_orig = np.zeros((nbins, 2, 2))
for i in range(nbins):
    cov_orig[i] = tilt.T.dot(cov_mod[i].dot(tilt))

print("\n ===== ESTIMATES FOR EACH BIN INCLUDING CORRELATIONS =====")

print("ln tau0", np.around(loc_orig[:, 0], 3))
print("gamma", np.around(loc_orig[:, 1], 3))

print("sig ln tau0", np.around(np.sqrt(cov_orig[:, 0, 0]), 3))
print("sig gamma", np.around(np.sqrt(cov_orig[:, 1, 1]), 3))

In [ ]:
print("\n ===== COMBINED ESTIMATES INCLDING SYSTEMATICS =====")

loc_7 = np.loadtxt("best_fit_loc_7.dat")
cov_7 = np.loadtxt("best_fit_cov_mat_7.dat")
loc_6 = np.loadtxt("best_fit_loc_6.dat")
cov_6 = np.loadtxt("best_fit_cov_mat_6.dat")

# The combined estimates in the original space
comb_loc_7 = xfm(loc_7, shift, tilt)[0]
comb_cov_7 = tilt.T.dot(cov_7.dot(tilt))
comb_loc_6 = xfm(loc_6, shift, tilt)[0]
comb_cov_6 = tilt.T.dot(cov_6.dot(tilt))

# Convert from ln tau0 estimates to tau0 estimates
t0t0_7 = np.exp(np.random.normal(comb_loc_7[0],
                np.sqrt(comb_cov_7[0, 0]), size=10000))
t0t0_6 = np.exp(np.random.normal(comb_loc_6[0],
                np.sqrt(comb_cov_6[0, 0]), size=10000))

print("\n ******* Including bin 7 *******")
print("tau_0 = %.5f pm %.5f" %(t0t0_7.mean(), t0t0_7.std()))
print("ln tau0 = %.5f pm %.5f" % (comb_loc_7[0], np.sqrt(comb_cov_7[0, 0])))
print("gamma = %.5f pm %.5f" % (comb_loc_7[1], np.sqrt(comb_cov_7[1, 1])))

print("\n ******* Excluding bin 7 *******")
print("tau_0 = %.5f pm %.5f" % (t0t0_6.mean(), t0t0_6.std()))
print("ln tau0 = %.5f pm %.5f" % (comb_loc_6[0], np.sqrt(comb_cov_6[0, 0])))
print("gamma = %.5f pm %.5f" % (comb_loc_6[1], np.sqrt(comb_cov_6[1, 1])))

print("\n ===== SYSTEMATIC MATRIX =====")
sys_7 = np.loadtxt(sfx + "_best_fit_sys_mat_7.dat")
sys_6 = np.loadtxt(sfx + "_best_fit_sys_mat_6.dat")

orig_sys_7 = tilt.T.dot(sys_7.dot(tilt))
orig_sys_6 = tilt.T.dot(sys_6.dot(tilt))

print("Systematic matrix including bin 7 \n", orig_sys_7)
print("Systematic matrix excluding bin 7 \n", orig_sys_6)

if plotit:
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', "#17becf"]

    fig, [ax1, ax2] = plt.subplots(ncols=2, figsize=(12, 4.5))

    # plot the contours for each bin
    ellipses = []
    for i in range(nbins):
        ellipses.append(plot_ellipse(loc_mod[i], cov_mod[i], set_label=False,
                                     nsig=[1, 2], ax=ax1, edgecolor=colors[i],
                                     facecolor='none'))

    # plot the contours for the combined estimate
    one_sig_7 = np.loadtxt(sfx + "_contour_one_sigma_7.dat")
    two_sig_7 = np.loadtxt(sfx + "_contour_two_sigma_7.dat")

    # bin7 + stst + sys contour
    ax1.plot(*one_sig_7.T, lw=1, color='k', ls="dashed")
    ax1.plot(*two_sig_7.T, lw=1, color='k', ls="dashed")

    one_sig_6 = np.loadtxt(sfx + "_contour_one_sigma_6.dat")
    two_sig_6 = np.loadtxt(sfx + "_contour_two_sigma_6.dat")

    # w/o bin7 + stst + sys contour
    ax1.plot(*one_sig_6.T, lw=1, color='k')
    ax1.plot(*two_sig_6.T, lw=1, color='k')

    # plot the systematic matrix
    # __ = plot_ellipse(loc_7, sys_7, nsig=[1], ax=ax1,
    #                   ec="brown", lw=2, ls="dashed", facecolor='none')
    # __ = plot_ellipse(loc_6, sys_6, nsig=[1], ax=ax1,
    #                   ec="purple", lw=2, ls="dashed", facecolor='none')

    ax1.set_xlim(-1.2, 2.2)
    ax1.set_ylim(-0.14, 0.08)
    ax1.set_xlabel(r"$x_0$")
    ax1.set_ylabel(r"$x_1$")

    handlers = [plt.plot([], ls="-", color=colors[i])[0] for i in range(7)]
    ax1.legend(ellipses, handles=handlers,
               labels=[r"$\mathrm{bin}\ %d$" % (i+1) for i in range(7)])

    # Convert to original space - this only gives points along the contour
    # so that we can draw it
    one_sig_orig_7 = xfm(one_sig_7, shift, tilt)
    two_sig_orig_7 = xfm(two_sig_7, shift, tilt)

    # bin7 + stat + sys
    ax2.plot(*one_sig_orig_7.T, lw=1, color='k', ls="dashed")
    ax2.plot(*two_sig_orig_7.T, lw=1, color='k', ls="dashed")

    one_sig_orig_6 = xfm(one_sig_6, shift, tilt)
    two_sig_orig_6 = xfm(two_sig_6, shift, tilt)

    # w/o bin 7 + stat + sys
    ax2.plot(*one_sig_orig_6.T, lw=1, color='k')
    ax2.plot(*two_sig_orig_6.T, lw=1, color='k')

    # TO PUT THE GRIDLINES FOR THE CHANGE OF BASIS
    x0_cons = np.linspace(-1, 2, 5)
    x00 = np.vstack((x0_cons, np.zeros(len(x0_cons)))).T
    X00 = xfm(x00, shift, tilt)

    x1_cons = np.linspace(-1, 2, 5)
    x11 = np.vstack((np.zeros(len(x1_cons)), x1_cons)).T
    X11 = xfm(x11, shift, tilt)

    ax2.plot(X00[:, 0], X00[:, 1], c="gray")
    ax2.plot(X11[:, 0], X11[:, 1], c="gray")

    # A simple weighted coadd of all the bins
    coadd_cov_orig_7 = np.linalg.inv(np.sum(np.linalg.inv(cov_orig), 0))

    temp = np.zeros(2)
    for i in range(7):
        temp += np.linalg.inv(cov_orig[i]).dot(loc_orig[i])

    coadd_loc_orig_7 = coadd_cov_orig_7.dot(temp)

    __ = plot_ellipse(coadd_loc_orig_7, coadd_cov_orig_7, nsig=[1, 2],
                      ax=ax2, ec="magenta", lw=1, facecolor='none')

    # hack to manually add the legend for the contours
    temp_lines = [plt.plot([], color="magenta",
                  label=r"$\mathrm{Statistical}$"),
                  plt.plot([], color="k", ls="dashed",
                  label=r"$\mathrm{Including\ systematics}$"),
                  plt.plot([], color="k",
                  label=r"$\mathrm{Including\ systematics\ w/o\ bin\ 7}$")]
    ax2.legend(temp_lines)

    # set axes limits
    ax2.set_xlim(-5.8, -4.7)
    ax2.set_ylim(2.85, 3.7)
    ax2.set_xlabel(r"$\ln \tau_0$")
    ax2.set_ylabel(r"$\gamma$")

    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.savefig("final_contours_orig.pdf")

    plt.show()